In [ ]:

"""
reads manual and automatic annotation from csv and calculates the center of the LM starttimes 

"""

In [ ]:
from os import walk
import matplotlib.transforms
import csv
import ast
import pyedflib
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy
from scipy.signal import oaconvolve
import time
import copy
import ast


In [ ]:
startfilename = "acq_000058160.edf.csv"
numberOfFiles = 6216
fileoffset = 0

AAnnopath = "C:\\Users\\aaron\\OneDrive\\Dokumente\\Uni\\Studienarbeit\\exampleAnnos\\"
MAnnopath = "C:\\Users\\aaron\\OneDrive\\Dokumente\\Uni\\Studienarbeit\\manualAnnos\\"

includepath = "C:\\Users\\aaron\\OneDrive\\Dokumente\\Uni\\Studienarbeit\\includedFiles.csv"
path = "Y:\\"
saveMetricdir = 'C:\\Users\\aaron\\OneDrive\\Dokumente\\Uni\\Studienarbeit\\finalMetrics\\'
###Detector
MAnnofilenames = next(walk(MAnnopath), (None, None, []))[2] 
#MAnnofilenames = [file[:-4] for file in MAnnofilenames]
AAnnofilenames = next(walk(AAnnopath), (None, None, []))[2] 
#AAnnofilenames = [file[:-4] for file in AAnnofilenames]
startAtFileNr = AAnnofilenames.index(startfilename)+ fileoffset
stopAtFileNr = startAtFileNr + numberOfFiles
#filenames = next(walk(path), (None, None, []))[2]
metricfilenames = next(walk(saveMetricdir), (None, None, []))[2] 

In [ ]:
#flattens any list to 1D, complicated due to not wanting to corrupt original list
def flattenx(nested_list, regular_list):
    
    for ele in nested_list:
        if type(ele) is list:
            flattenx(ele, regular_list)
        else:
            regular_list.append(ele)
    return regular_list

def flatten(nested_list):
    return flattenx(nested_list, regular_list = [])

In [ ]:
#only read the files that have manual annos
with open(includepath , "r", newline='') as csvfile:

    content = csv.reader(csvfile,  delimiter=';', quotechar='"')
    strings = list(content)
    data = []
    for e in strings:
        data.append(e)
includefiles = flatten(data)

In [ ]:
mCenters = []
aCenters = []
totalstarttime = time.time()

for filename in [file for file in includefiles[startAtFileNr:stopAtFileNr]]:
    filestarttime = time.time()
    sampleFreq = 200

    #reading csv back to list
    with open(AAnnopath + filename + ".csv" , "r") as csvfile:

        content = csv.reader(csvfile, delimiter=',', quotechar='"')
        strings = list(content)
        if(len(strings) == 0):
            continue
        aAnno = []
        for e in strings[0]:
            aAnno.append(eval(e))

    with open(MAnnopath + filename + ".csv" , "r") as csvfile:
        content = csv.reader(csvfile, delimiter=',', quotechar='"')
        strings = list(content)
        if(len(strings) == 0 or strings[0][0] == " "):
            continue
        mAnno = []
        for e in strings[0]:
            mAnno.append(eval(e))
            
    #centers
    mLMStarts = [LM[0] for LM in mAnno]
    mCenter = sum(mLMStarts)/len(mLMStarts) if len(mLMStarts) != 0  else math.nan
    mCenters.append(mCenter)
    
    aLMStarts = [LM[0] for LM in aAnno]
    aCenter = sum(aLMStarts)/len(aLMStarts) if len(aLMStarts) != 0  else math.nan
    aCenters.append(aCenter)


In [ ]:
centerdiffs = [aCenter - mCenter for (mCenter,aCenter) in zip(mCenters,aCenters) if mCenter != math.nan and  aCenter != math.nan]
signal = [x for x in centerdiffs if x != math.inf]
bins = "auto"
print(np.nanmean(signal)/200)
y,x,_ = plt.hist(signal, bins = bins)

plt.title("Differenz der Schwerpunkte aus \n automatischer Annotation und manueller Annotation")
plt.xlabel("Abweichung in Samples")

plt.ylabel("Anzahl der Annotationen")
print(x[np.argmax(y)]/200)
print(np.nanmedian(signal)/200)
plt.savefig("centers"+".jpg", dpi = 400.0)

savepath = "C:\\Users\\aaron\\OneDrive\\Dokumente\\Uni\\Studienarbeit\\centerDiffs.csv"
with open(savepath,'w') as file:
    for l in centerdiffs:
        file.write(str(l))
        file.write('\n')